In [1]:
# import dependencies
import pandas as pd
import numpy as np
import os

# creating path to CSV output
povertyCSV = (os.path.join('povertyCSV.csv'))

In [2]:
# pull in original CSV from FRED
# CSV pulled from https://geofred.stlouisfed.org/map/?th=pubugn&cc=5&rc=false&im=fractile&sb&lng=-49.6&lat=45.5&zm=2&sl&sv&am=Average&at=Not%20Seasonally%20Adjusted,%20Annual,%20Percent&dt=2015-01-01&fq=Annual&rt=county&sti=150203&un=lin
csvPath = 'Poverty%ByUSCounty.csv'
povData = pd.read_csv(csvPath)

# preview CSV
#povData.head()

In [3]:
# turning CSV into a dataframe
povDF = pd.DataFrame(povData, columns = ['County', 'State', '2012', '2013', '2014', '2015', '2016', '2017'])

In [4]:
# create a dictionary for each of the state abbreviations so this can be merged with CDC data flawlessly
abbrevToFullName = {'AL': 'Alabama', 'MT': 'Montana' , 'AK': 'Alasaka', 'NE': 'Nebraska',
                    'AZ': 'Arizona', 'NV': 'Nevada','AR': 'Arkansas', 'NH': 'New Hampshire',
                    'CA': 'California', 'NJ': 'New Jersey', 'CO': 'Colorado', 'NM': 'New Mexico',
                    'CT': 'Connecticut', 'NY': 'New York', 'DE': 'Delaware', 'NC': 'North Carolina',
                    'FL': 'Florida', 'ND': 'North Dakota', 'GA': 'Georgia', 'OH': 'Ohio',
                    'HI': 'Hawaii', 'OK': 'Oklahoma', 'ID': 'Idaho', 'OR': 'Oregon', 'IL': 'Illinois',
                    'PA': 'Pennsylvania', 'IN': 'Indiana', 'RI': 'Rhode Island', 'IA': 'Iowa', 'SC': 'South Carolina',
                    'KS': 'Kansas', 'SD': 'South Dakota', 'KY': 'Kentucky', 'TN': 'Tennessee', 'LA': 'Louisiana',
                    'TX': 'Texas', 'ME': 'Maine', 'UT': 'Utah', 'MD': 'Maryland', 'VT': 'Vermont', 
                    'MA': 'Massachusetts', 'VA': 'Virginia', 'MI': 'Michigan', 'WA': 'Washington',
                    'MN': 'Minnesota', 'WV': 'West Virginia', 'MS': 'Mississippi', 'WI': 'Wisconsin', 
                    'MO': 'Missouri', 'WY': 'Wyoming'}

# add a space in front of each key in the dictionary, original CSV has a SPACE before each abbreviation
abbrevToFullNameWithSpaces = {(" " + abbrev):full for abbrev, full in abbrevToFullName.items()}

# show adjusted dictionary with spaces added
print(abbrevToFullNameWithSpaces)

{' AL': 'Alabama', ' MT': 'Montana', ' AK': 'Alasaka', ' NE': 'Nebraska', ' AZ': 'Arizona', ' NV': 'Nevada', ' AR': 'Arkansas', ' NH': 'New Hampshire', ' CA': 'California', ' NJ': 'New Jersey', ' CO': 'Colorado', ' NM': 'New Mexico', ' CT': 'Connecticut', ' NY': 'New York', ' DE': 'Delaware', ' NC': 'North Carolina', ' FL': 'Florida', ' ND': 'North Dakota', ' GA': 'Georgia', ' OH': 'Ohio', ' HI': 'Hawaii', ' OK': 'Oklahoma', ' ID': 'Idaho', ' OR': 'Oregon', ' IL': 'Illinois', ' PA': 'Pennsylvania', ' IN': 'Indiana', ' RI': 'Rhode Island', ' IA': 'Iowa', ' SC': 'South Carolina', ' KS': 'Kansas', ' SD': 'South Dakota', ' KY': 'Kentucky', ' TN': 'Tennessee', ' LA': 'Louisiana', ' TX': 'Texas', ' ME': 'Maine', ' UT': 'Utah', ' MD': 'Maryland', ' VT': 'Vermont', ' MA': 'Massachusetts', ' VA': 'Virginia', ' MI': 'Michigan', ' WA': 'Washington', ' MN': 'Minnesota', ' WV': 'West Virginia', ' MS': 'Mississippi', ' WI': 'Wisconsin', ' MO': 'Missouri', ' WY': 'Wyoming'}


In [5]:
# map full state name into the CSV by pairing full state name from dictionary with the state abbreviation
povDF['Full State Name'] = povDF['State'].map(abbrevToFullNameWithSpaces)
# preview dataframe with fullstate added
povDF.head()

,County,State,2012,2013,2014,2015,2016,2017,Full State Name
0,"Aleutians East Borough, AK",AK,16.5,16.7,16.4,16.8,15.5,16.7,Alasaka
1,"Aleutians West CA, AK",AK,10.5,9.2,8.9,9.1,7.7,7.5,Alasaka
2,"Anchorage Muny, AK",AK,7.7,7.9,8.3,8.2,8.1,8.1,Alasaka
3,"Bethel CA, AK",AK,21.8,22.8,23.7,25.2,26.6,27.0,Alasaka
4,"Bristol Bay Borough, AK",AK,6.2,7.9,7.2,7.2,7.3,7.1,Alasaka


In [6]:
# set up what we want to new order to be in the revised dataframe
newOrder = ['Full State Name', 'State', 'County', '2012', '2013', '2014', '2015', '2016', '2017']
# reorder columns in new data frame
povertyRates = povDF.reindex(columns=newOrder)
# export to CSV
povertyRates.to_csv(os.path.join('povertyCSV.csv'))
# preview final dataFrame
povertyRates.head()

,Full State Name,State,County,2012,2013,2014,2015,2016,2017
0,Alasaka,AK,"Aleutians East Borough, AK",16.5,16.7,16.4,16.8,15.5,16.7
1,Alasaka,AK,"Aleutians West CA, AK",10.5,9.2,8.9,9.1,7.7,7.5
2,Alasaka,AK,"Anchorage Muny, AK",7.7,7.9,8.3,8.2,8.1,8.1
3,Alasaka,AK,"Bethel CA, AK",21.8,22.8,23.7,25.2,26.6,27.0
4,Alasaka,AK,"Bristol Bay Borough, AK",6.2,7.9,7.2,7.2,7.3,7.1
